In [151]:
# from google.colab import auth
# from googleapiclient.discovery import build
# from io import FileIO
# from googleapiclient.http import MediaIoBaseDownload

import pandas as pd
import nltk
from sklearn.model_selection import train_test_split

try:
  stop_words = nltk.corpus.stopwords.words("english")
except:
  nltk.download('stopwords')
  stop_words = nltk.corpus.stopwords.words("english")

import numpy as np
import torch
import torch.nn as nn
# import matplotlib.pyplot as plt
# %matplotlib inline

In [152]:
#  Get the files from google drive
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Get english train data file
file_id = '1m3Ax9Z8OHMU-7FqraKc-ddI3YQ7yY_Q6'  # file id on the Google Drive
downloaded = FileIO("en.trial.complete.json", 'w')
request = drive_service.files().get_media(fileId=file_id)
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  status, done = downloader.next_chunk()
  print("Download {}%.".format(int(status.progress() * 100)))

NameError: name 'auth' is not defined

In [153]:
en_df = pd.read_json("trial-data_all/en.trial.complete.json")
en_df.head()

,id,word,pos,gloss,example,type,counts,f_rnk,concrete,polysemous,sgns,char,electra
0,en.trial.1,beautiful,adjective,Pleasant ; clear .,"It 's beautiful outside , let 's go for a walk .",synonym/antonym-based,124908,706,0,0,"[1.393769145, 0.7516670227000001, -2.581333160...","[0.295645088, 0.098426342, 0.0463486575, 0.016...","[0.0800914839, -0.1875839084, -0.0411579385000..."
1,en.trial.2,cocktail,noun,A mixture of other substances or things .,a cocktail of illegal drugs,hypernym-based,4187,13245,1,0,"[2.0872907639, 0.2617726326, 0.668431639700000...","[0.3878918886, 0.1971583217, -0.44026631120000...","[-1.4771454334, -0.4742421806, 0.0847439319, -..."
2,en.trial.3,institutionalized,adjective,Having been established as an institution .,It is very difficult to get bureaucracies to a...,paraphrastic,961,35934,0,0,"[0.7893871069, -0.43510755900000003, 0.8553860...","[-0.0519028902, 0.2257766128, -0.1839749813, 0...","[-1.1030955315, -0.9046602845, 0.1503403783, -..."
3,en.trial.4,menial,noun,"A servant , especially a domestic servant .","The world was awake to the 2nd of May , but Ma...",hypernym-based,517,53267,1,1,"[0.1222261563, 0.1572209597, 0.5396134257, -0....","[-0.3667449057, -0.1431699395, -0.0671329796, ...","[-1.6584062576, -0.24498166140000002, 0.150174..."
4,en.trial.5,seek,verb,To try to find ; to look for ; to search for .,"Not long ago , it was difficult to produce pho...",paraphrastic,25195,3212,0,0,"[1.1894155741, 1.3668279648000001, -1.61634504...","[0.6137102246, 0.5464909673, -0.0161557049, 9....","[-0.5474479198000001, -0.0880863219, 0.0784259..."


In [154]:
def clean(gloss):
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  cleaned = tokenizer.tokenize(gloss)
  cleaned = list(set([word.lower() for word in cleaned]))
  cleaned = [word for word in cleaned if not word in stop_words]
  return cleaned

gloss_lists = en_df.gloss.apply(clean)
gloss_lists

0                                      [clear, pleasant]
1                          [substances, things, mixture]
2                             [established, institution]
3                        [domestic, especially, servant]
4                              [look, search, find, try]
                             ...                        
195                        [color, cells, animal, plant]
196                                [seeming, appearance]
197       [proceed, travel, person, permission, vehicle]
198                      [much, sitting, moving, around]
199    [sentence, heraldic, word, forming, achievemen...
Name: gloss, Length: 200, dtype: object

In [155]:
# list of all context words
context_voc = []
for i in range(len(gloss_lists)):
  for j in range(len(gloss_lists[i])):
    if not gloss_lists[i][j] in context_voc:
      context_voc.append(gloss_lists[i][j])

In [156]:
# true context words for each defined word (center word)
trues = []
for i in range(len(gloss_lists)):
  for j in range(len(gloss_lists[i])):
    index = context_voc.index(gloss_lists[i][j])
    # append index of center in gloss_lists, index in context_voc, and 1 for true
    trues.append([i, index, 1])


In [163]:
# mostly follows this: https://adoni.github.io/2017/11/08/word2vec-pytorch/
class SGNS(nn.Module):
    def __init__(self) -> 'SGNS':
        super(SGNS, self).__init__()
        self.center_embs = nn.Embedding(len(gloss_lists), 256, sparse = True)
        self.context_embs = nn.Embedding(len(context_voc), 256, sparse = True)
        nn.init.xavier_uniform_(self.center_embs.weight)
        nn.init.xavier_uniform_(self.context_embs.weight)
    def forward(self, center:'center index', pos_con:'trues index', neg_cons:'list of falses indices')->'scores':
        scores = []
        center_emb = self.center_embs(torch.LongTensor([center]))
        pos_context_emb = self.context_embs(torch.LongTensor([pos_con]))
        pos_score = torch.mul(center_emb, pos_context_emb).squeeze()
        pos_score = torch.sum(pos_score)
        pos_score = nn.functional.logsigmoid(pos_score)
        scores.append(pos_score)

        for neg_con in neg_cons:
            neg_context_emb = self.context_embs(torch.LongTensor([neg_con]))
            neg_score = torch.mul(center_emb, neg_context_emb).squeeze()
            neg_score = torch.sum(neg_score)
            neg_score = nn.functional.logsigmoid(-1 * neg_score)
            scores.append(neg_score)

        return -1 * sum(torch.stack(scores))
        
sgns = SGNS()
sgns

SGNS(
  (center_embs): Embedding(200, 256, sparse=True)
  (context_embs): Embedding(769, 256, sparse=True)
)

In [164]:
optim = torch.optim.SparseAdam(sgns.parameters(), lr = .001)

# number of negative pairs for each train. 2 - 5 usually works well for large datasets
neg_count = 3
epochs = 100

for i in range(epochs):
    for j in range(len(gloss_lists)):
        optim.zero_grad()
        # to add: choice distribution based on frequency raised to 3/4 power
        falses = np.random.choice(len(context_voc), neg_count, replace = False)
        loss = sgns.forward(trues[i][0], trues[i][1], falses)
        loss.backward()
        optim.step()
    if i % 10 == 0:
        print(loss.data)
print(loss.data)

tensor(1.9947)
tensor(0.5099)
tensor(0.2892)
tensor(0.0315)
tensor(0.0481)
tensor(0.0313)
tensor(0.0370)
tensor(0.1617)
tensor(0.0303)
tensor(0.2597)
tensor(0.0151)


In [165]:
embeddings = sgns.center_embs.weight.tolist()
print(len(embeddings), len(embeddings[0]))

200 256


In [166]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, en_df.sgns.values)
print(X_train[0][:5], "\n", y_train[0][:5])

[0.09188833087682724, 0.0751756802201271, 0.07847709208726883, 0.06864740699529648, -0.0531768761575222] 
 [-0.4988436103, -0.6650734544, -1.3417009115, 0.5371951461, -0.7869545221]
